In [1]:
conda install -c anaconda beautifulsoup4 


Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\tmadan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB  anaconda
    anaconda-custom            |           py37_1           3 KB
    beautifulsoup4-4.8.0       |           py37_0         147 KB  anaconda
    ca-certificates-2019.5.15  |                1         166 KB  anaconda
    certifi-2019.6.16          |           py37_1         156 KB  anaconda
    openssl-1.1.1              |       he774522_0         5.7 MB  anaconda
    tbb

In [3]:
pip install tinydb urllib3 xlsxwriter lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import datetime
from tinydb import TinyDB, Query
import urllib3
import xlsxwriter

### Parse Wikipedia URL

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
total_added = 0

In [6]:
def make_soup(url):
    http = urllib3.PoolManager()
    r = http.request("GET", url)
    return BeautifulSoup(r.data,'lxml')

In [7]:
results = make_soup(url)

In [22]:
required_Json = results.find("table", class_="wikitable sortable")

In [56]:
postcode=[]
borough=[]
neighbourhood=[]

for tr in required_Json.find_all('tr'):
    tds=tr.find_all('td')    
    if(len(tds)==3):
          postcode.append(tds[0].text);
          borough.append(tds[1].text);
          neighbourhood.append(tds[2].text.replace('\n',''))
          


   

In [51]:
import pandas as pd


In [74]:
d = {'Postcode':postcode,'Borough':borough, 'Neighbourhood': neighbourhood}
df = pd.DataFrame(d)
df[0:5]


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [75]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

In [77]:
df.shape

(211, 3)

In [80]:
df['Borough'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### Assign Borough as Neighbourhood if Neighbourhood not assigned

In [99]:
df['Neighbourhood'].value_counts()

Runnymede               2
St. James Town          2
CN Tower                1
The Queensway East      1
Wexford                 1
South Hill              1
The Queensway West      1
The Junction South      1
The Annex               1
Birch Cliff             1
Mimico South            1
Silver Hills            1
Forest Hill North       1
Port Union              1
Rouge                   1
Martin Grove            1
West Deane Park         1
Woodbine Gardens        1
Little Portugal         1
Upwood Park             1
The Junction North      1
Dorset Park             1
Markland Wood           1
Mimico NW               1
Bathurst Manor          1
Weston                  1
Beaumond Heights        1
The Beaches West        1
Cliffside West          1
Islington               1
                       ..
Berczy Park             1
Guildwood               1
South Niagara           1
Eringate                1
East Toronto            1
Steeles East            1
Martin Grove Gardens    1
Albion Garde

In [98]:
df1 = df.loc[df['Neighbourhood']=='Not assigned'].index
df1

Int64Index([], dtype='int64')

In [95]:
df.loc[df1, 'Neighbourhood']=df.loc[df1]['Borough']
df.loc[df1]


,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


### Merge Neigbourhood basis same postcode

In [135]:
df_updated = df['Neighbourhood'].groupby([df.Postcode, df.Borough]).apply(','.join).reset_index()
df_updated

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [136]:
#read lat and Long from csv file
url="http://cocl.us/Geospatial_data"
position_df=pd.read_csv(url)

In [138]:
position_df.shape

(103, 3)

In [139]:
position_df[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [165]:
position_df.rename(columns={"Postal Code": "Postcode"},inplace=True)
#position_df[0:5]
#df_updated[0:5]
new_df = pd.merge(df_updated, position_df, on='Postcode')
new_df[0:5]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
